# Phase-1: Skin Disease Detection – EDA & Preprocessing
**Project:** Skin Lesion Classification using CNN + XAI  
**Environment:** (skinproj) Python 3.10


In [4]:
print(df_images.columns)
df_images.head()


NameError: name 'df_images' is not defined

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

%matplotlib inline
plt.rcParams['figure.figsize'] = (8,5)
sns.set(style="whitegrid")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Dataset Setup

Place your HAM10000 dataset in this structure:

data/
raw/
images/ ← all merged images here
HAM10000_metadata.csv

bash
Copy code

If you have only a few sample images, place them in `data/raw/images/`.

In [ ]:
data_dir = Path("data/raw")
meta_path = data_dir / "HAM10000_metadata.csv"

if meta_path.exists():
    meta = pd.read_csv(meta_path)
    print("Metadata loaded:", meta.shape)
    display(meta.head())
else:
    print("No metadata file found.")
    meta = None


In [4]:
image_paths = []
labels = []

images_folder = data_dir / "images"

for img_path in images_folder.glob("*.*"):
    if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:
        image_paths.append(str(img_path))
        labels.append("unknown")   # no class label in HAM10000 by default

df_images = pd.DataFrame({
    "image_path": image_paths,
    "label": labels
})

print("Total images found:", len(df_images))
df_images.head()


Total images found: 10015


,image_path,label
0,data\raw\images\ISIC_0024306.jpg,unknown
1,data\raw\images\ISIC_0024307.jpg,unknown
2,data\raw\images\ISIC_0024308.jpg,unknown
3,data\raw\images\ISIC_0024309.jpg,unknown
4,data\raw\images\ISIC_0024310.jpg,unknown


In [3]:
image_paths = []
labels = []

images_folder = data_dir / "images"

for img_path in images_folder.glob("*.*"):
    if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:
        image_paths.append(str(img_path))
        labels.append("unknown")   # no class label in HAM10000 by default

df_images = pd.DataFrame({
    "image_path": image_paths,
    "label": labels
})

print("Total images found:", len(df_images))
df_images.head()


NameError: name 'data_dir' is not defined

In [5]:
def show_samples(df, n=9):
    sample = df.sample(min(n, len(df)), random_state=42)
    fig, axs = plt.subplots(3, 3, figsize=(12,12))
    axs = axs.flatten()
    
    for ax, (_, row) in zip(axs, sample.iterrows()):
        img = cv2.imread(row['image_path'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img)
        ax.set_title(row['label'])
        ax.axis('off')
    
    plt.tight_layout()

show_samples(df_images)


sys.executable: C:\Users\prowo\.conda\envs\skinproj\python.exe
python: 3.10.19
matplotlib: 3.6.3
matplotlib_inline import error: AttributeError("'RcParams' object has no attribute '_get'")


In [7]:
plt.figure(figsize=(6,3))
sns.countplot(data=df_images, x="label")
plt.title("Class Distribution (placeholder)")
plt.tight_layout()


AttributeError: 'RcParams' object has no attribute '_get'

In [ ]:
if meta is not None:
    display(meta.describe(include='all').T)
    
    if "age" in meta.columns:
        plt.figure(figsize=(8,4))
        sns.histplot(meta['age'].dropna(), bins=20)
        plt.title("Age Distribution")

    if "sex" in meta.columns:
        plt.figure(figsize=(6,3))
        sns.countplot(x="sex", data=meta)
        plt.title("Sex Distribution")

    if "localization" in meta.columns:
        plt.figure(figsize=(10,4))
        sns.countplot(y="localization", data=meta,
                      order=meta["localization"].value_counts().index)
        plt.title("Lesion Localization")
else:
    print("Metadata not available.")


In [ ]:
def preprocess_image(path, size=(224,224)):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(img, size)
    normalized = resized / 255.0
    return resized, normalized

sample = df_images.sample(3)

fig, axs = plt.subplots(2,3, figsize=(12,7))
axs = axs.flatten()

for i, (_, row) in enumerate(sample.iterrows()):
    original = cv2.cvtColor(cv2.imread(row["image_path"]), cv2.COLOR_BGR2RGB)
    resized, norm = preprocess_image(row["image_path"])

    axs[2*i].imshow(original)
    axs[2*i].set_title("Original")
    axs[2*i].axis("off")
    
    axs[2*i+1].imshow(resized)
    axs[2*i+1].set_title("Resized 224x224")
    axs[2*i+1].axis("off")

plt.tight_layout()


In [ ]:
import random

def augment_image(img):
    if random.random() > 0.5:
        img = cv2.flip(img, 1)
    angle = random.uniform(-25, 25)
    M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), angle, 1)
    img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    return img

sample = df_images.sample(3)

fig, axs = plt.subplots(2,3, figsize=(12,7))
axs = axs.flatten()

for i, (_, row) in enumerate(sample.iterrows()):
    img = cv2.cvtColor(cv2.imread(row["image_path"]), cv2.COLOR_BGR2RGB)
    aug = augment_image(img.copy())

    axs[2*i].imshow(img)
    axs[2*i].set_title("Original")
    axs[2*i].axis("off")

    axs[2*i+1].imshow(aug)
    axs[2*i+1].set_title("Augmented")
    axs[2*i+1].axis("off")

plt.tight_layout()


In [ ]:
out = Path("outputs")
out.mkdir(exist_ok=True, parents=True)

# Save resized sample image
p = df_images.iloc[0]["image_path"]
resized, _ = preprocess_image(p)
Image.fromarray(resized).save(out/"resized_example.png")

# Save class distribution
plt.figure(figsize=(6,3))
sns.countplot(data=df_images, x="label")
plt.title("Class Distribution")
plt.tight_layout()
plt.savefig(out/"class_distribution.png", dpi=150)

print("Saved output images to outputs/ folder.")


In [ ]:
# CELL A: Build labeled dataframe by matching image_id in metadata to files
from pathlib import Path
import pandas as pd

data_dir = Path("data/raw")
images_dir = data_dir / "images"
meta_path = data_dir / "HAM10000_metadata.csv"

if meta_path.exists():
    meta = pd.read_csv(meta_path)
    img_to_label = {}
    for _, r in meta.iterrows():
        imgid = str(r['image_id'])
        # try common extensions
        for ext in (".jpg", ".jpeg", ".png"):
            p = images_dir / (imgid + ext)
            if p.exists():
                img_to_label[str(p)] = str(r['dx'])
                break

    rows = []
    for p in images_dir.glob("*.*"):
        fp = str(p)
        label = img_to_label.get(fp, None)
        if label is None:
            continue
        rows.append((fp, label))
    df_lab = pd.DataFrame(rows, columns=["image_path", "label"])
    print("Labeled images from metadata:", df_lab.shape)
    display(df_lab.label.value_counts())
else:
    print("Metadata missing at:", meta_path)
    df_lab = None


In [ ]:
# CELL B: Create a small balanced subset for fast demo training
from sklearn.model_selection import train_test_split

# Tune this: how many images per class for demo training
N_per_class = 80   # change to 40 or 50 if you want it even faster

# Build small balanced sample
samples = []
for cls, grp in df_lab.groupby('label'):
    n = min(len(grp), N_per_class)
    samples.append(grp.sample(n, random_state=1))
df_small = pd.concat(samples).sample(frac=1, random_state=1).reset_index(drop=True)

# Stratified train/validation split
train_df, val_df = train_test_split(df_small, test_size=0.2, stratify=df_small['label'], random_state=1)

print("Selected N_per_class:", N_per_class)
print("df_small shape:", df_small.shape)
print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)
print("\nClass counts in df_small:\n", df_small['label'].value_counts())


In [ ]:
# CELL C: Create Keras ImageDataGenerators from dataframe
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224,224)
BATCH_SIZE = 16   # reduce to 8 if memory issues

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    train_df, x_col='image_path', y_col='label',
    target_size=IMG_SIZE, class_mode='categorical',
    batch_size=BATCH_SIZE, shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df, x_col='image_path', y_col='label',
    target_size=IMG_SIZE, class_mode='categorical',
    batch_size=BATCH_SIZE, shuffle=False
)

class_indices = train_gen.class_indices
inv_class_indices = {v:k for k,v in class_indices.items()}

print("Classes (name -> index):", class_indices)
print("Inverse mapping (index -> name):", inv_class_indices)


In [ ]:
# CELL D: Build transfer-learning model (ResNet50 base frozen)
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

num_classes = len(class_indices)   # should be 7

# load ResNet50 base (imagenet weights), exclude top
base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base.trainable = False   # freeze the convolutional base for fast demo

# add classification head
x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base.input, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Number of classes:", num_classes)
model.summary()


In [ ]:
# CELL E: Train model for a few epochs (demo)
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from pathlib import Path
import matplotlib.pyplot as plt

# make outputs folder
Path("outputs").mkdir(exist_ok=True)

# training hyperparams — tune if needed
EPOCHS = 4           # change to 2-3 if you want it very fast
BATCH_SIZE = 16      # matches the generator batch size
STEPS_PER_EPOCH = len(train_gen)  # derived from generator
VALIDATION_STEPS = len(val_gen)

print("Epochs:", EPOCHS, "Batch size:", BATCH_SIZE)
print("Steps per epoch:", STEPS_PER_EPOCH, "Validation steps:", VALIDATION_STEPS)

# Callbacks
ckpt_path = "outputs/resnet50_demo_best.h5"
ckpt = ModelCheckpoint(ckpt_path, monitor='val_accuracy', save_best_only=True, verbose=1)
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
es   = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    callbacks=[ckpt, rlrp, es]
)

# Save history for plotting in next cell
import pickle
with open("outputs/history.pkl", "wb") as f:
    pickle.dump(history.history, f)

print("Training finished. Best model (if any) saved to:", ckpt_path)


In [ ]:
# CELL F (save-to-file fallback) — safe if inline backend is broken
import pickle, matplotlib
matplotlib.use('Agg')   # use non-interactive backend to avoid inline issues
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras.models import load_model
import IPython.display as disp

hist_path = Path("outputs/history.pkl")
if hist_path.exists():
    with open(hist_path, "rb") as f:
        history = pickle.load(f)
else:
    try:
        history = history.history
    except Exception as e:
        print("No history found:", e)
        history = None

if history:
    fig = plt.figure(figsize=(10,4))
    ax1 = fig.add_subplot(1,2,1)
    ax1.plot(history.get('loss', []), label='train loss')
    ax1.plot(history.get('val_loss', []), label='val loss')
    ax1.legend(); ax1.set_title('Loss')

    ax2 = fig.add_subplot(1,2,2)
    ax2.plot(history.get('accuracy', []), label='train acc')
    ax2.plot(history.get('val_accuracy', []), label='val acc')
    ax2.legend(); ax2.set_title('Accuracy')

    out_path = Path("outputs/training_curves.png")
    fig.savefig(out_path, bbox_inches='tight')
    plt.close(fig)
    print("Saved training curves to:", out_path)
    display(disp.Image(str(out_path)))
else:
    print("No history to plot.")

best_path = Path("outputs/resnet50_demo_best.h5")
if best_path.exists():
    model = load_model(str(best_path))
    print("Loaded best model from:", best_path)
else:
    print("Best model not found; continuing with current model object.")


In [ ]:
# SAFE DISPLAY: Grad-CAM -> save images to disk and display (no matplotlib inline required)
import numpy as np
import cv2
from pathlib import Path
import tensorflow as tf
from IPython.display import Image, display

# reuse display_gradcam logic but save files
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

def save_and_show_gradcam(image_path, model, inv_class_indices, IMG_SIZE=(224,224), alpha=0.4, last_conv_layer_name=None):
    img = cv2.imread(str(image_path))
    if img is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, IMG_SIZE)
    img_input = img_resized.astype(np.float32) / 255.0
    img_tensor = np.expand_dims(img_input, axis=0)

    # auto-find last conv layer if not given
    if last_conv_layer_name is None:
        last_conv = None
        for layer in reversed(model.layers):
            out_shape = getattr(layer, "output_shape", None)
            if out_shape and len(out_shape) == 4:
                last_conv = layer.name
                break
        if last_conv is None:
            raise ValueError("Could not find a 4D conv layer in the model; pass last_conv_layer_name manually.")
        last_conv_layer_name = last_conv

    print("Using last conv layer:", last_conv_layer_name)

    heatmap = make_gradcam_heatmap(img_tensor, model, last_conv_layer_name)
    heatmap_resized = cv2.resize(heatmap, (img_rgb.shape[1], img_rgb.shape[0]))
    heatmap_uint8 = np.uint8(255 * heatmap_resized)
    heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    heatmap_color_rgb = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB)
    overlay = cv2.addWeighted(img_rgb.astype(np.uint8), 1-alpha, heatmap_color_rgb, alpha, 0)

    # predictions
    preds = model.predict(np.expand_dims(img_input, axis=0))
    pred_idx = int(np.argmax(preds[0]))
    pred_label = inv_class_indices[pred_idx]
    pred_conf = float(preds[0][pred_idx])

    # make outputs folder
    out = Path("outputs")
    out.mkdir(exist_ok=True)

    orig_path = out / "gradcam_original.png"
    heat_path = out / "gradcam_heatmap.png"
    over_path = out / "gradcam_overlay.png"

    # save (convert RGB -> BGR for cv2.imwrite)
    cv2.imwrite(str(orig_path), cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))
    cv2.imwrite(str(heat_path), cv2.cvtColor(heatmap_color_rgb, cv2.COLOR_RGB2BGR))
    cv2.imwrite(str(over_path), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

    print(f"Predicted: {pred_label}  Confidence: {pred_conf:.4f}")
    print("Saved files:\n", orig_path, "\n", heat_path, "\n", over_path)
    display(Image(str(orig_path)))
    display(Image(str(heat_path)))
    display(Image(str(over_path)))
    return pred_label, pred_conf

# pick a random validation image and run
test_row = val_df.sample(1).iloc[0]
test_image_path = test_row['image_path']
print("Test image (true label):", test_row['label'])
save_and_show_gradcam(test_image_path, model, inv_class_indices)


## Phase-2 (Upcoming)
- Full CNN Model Training  
- Grad-CAM Heatmaps  
- LIME & SHAP Explainability  
- Augmentation Improvement  
- Evaluation Metrics  
- Final Deployment (Flask Web App)
